# Generate a Reactions file

This notebook generates a Spud XML file for an Olivine-Liquid melting reaction with a Forsterite-Fayalite ideal binary solid solution

standard imports

In [ ]:
import sympy as sym
import os
from glob import glob
sym.init_printing()

import ThermoCodegen packages

In [ ]:
from thermocodegen.codegen import reaction
from thermocodegen.coder import coder

Get the working version of ThermoCodegen

In [ ]:
import thermocodegen as tcg
tcg_version = tcg.__version__
print('Using ThermoCodegen v{}'.format(tcg_version))

Set a reference string for this Notebook

In [ ]:
reference = 'Thermocodegen-v{}/examples/Systems/fo_fa/notebooks/Generate_reactions.ipynb'.format(tcg_version)

List of phases present in this set of reactions

In [ ]:
phase_names = ['Olivine', 'Liquid']

Path to the thermodynamic database tarball file that this set of reactions are built on

Here we use a relative path to the compressed database file.  This can lead to issues if the final rxml file is moved so an absolute path to a local file is safer, however, it also makes the rxml files less easy to share among different users.

In [ ]:
db = '../database/fo_fa.tar.gz'

Alternatively, we could use a remote thermodynamic database, for example the one available as a DOI from zenodo 

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.6350797.svg)](https://doi.org/10.5281/zenodo.6350797)


In [ ]:
#db = 'https://zenodo.org/record/6350797/files/fo_fa.tar.gz'

Instantiate a Reaction object. This is initialized using the 'from_database' class method. Initializing in this way requires the total number of reaction (total_reactions), a list of phase names (phase_names) and a path to a thermocodegen generated thermodynamic database.

In [ ]:
rxn = reaction.Reaction.from_database(total_reactions=2, phase_names=phase_names, database=db)

The 'model_dict' attribute of this object contains the current state of the information that we have given and what has been extracted from the thermodynamic database 'fo_fa.tar.gz'

In [ ]:
rxn.model_dict

Set some utility indices for referencing phases and endmembers (this should be automated somehow)

In [ ]:
iOl = 0
iLiq = 1
kFo = 0
kFa = 1

## Define the reactions in this system

and then write SymPy that describes the reactions rates

First we pull out the SymPy symbols for the independent variables (temperature $T$, pressure $P$, concentration $C$ and phase fraction $\Phi$). Concentration C is a `sym.MatrixSymbol` of dimension N (number of phases) by Kmax (maximum number of endmembers). Phase fraction Phi is a `sym.MatrixSymbol` of dimension N (number of phases) by 1 (i.e., a vector).

In [ ]:
# Variables
T = rxn.T
P = rxn.P
C = rxn.C
Phi = rxn.Phi

Now we pull out the special symbol reserved for the affinity, A. This is a sym.MatrixSymbol of dimension J (number of reactions) by 1.

In [ ]:
# Affinity
A = rxn.A

Now we create  the parameters and sympy symbols used in the reaction rate expressions.  We will user `thermocodegen.coder.set_coder_params` to build the parameter tuples from strings and units
Then make the symbols available to the local dictionary to be used in other expressions.

For these models we only need two new parameters, a reference Arhennius temperature $T_0$ and the gas constant $R$

In [ ]:
param_strings = ['T0','R']
units = ['K','J/(mol K)']
params = coder.set_coder_params(param_strings,units)
symbol_dict = coder.get_symbol_dict_from_params(params)

locals().update(symbol_dict)
params

### Forsterite melting reaction

Set up the forsterite melting reaction (Forsterite_berman -> Forsterite_xmelts). This requires a list of tuples for both the reactants and products. Each tuple consists of strings of 

(name,phase,endmember)

where 
    **name**:  is an arbitrary name describing the reactant endmenber,
    **phase**: is the phase containing the endmember
    **endmember**: the name of the endmember that is reacting in that phase
    
The phase and endmember names should be consistent with whatever they are called in the thermodynamic database.

In [ ]:
# reaction 0
j = 0

# Reactants
Fo_Ol = ('Fo_Ol','Olivine','Forsterite_berman')

# Products
Fo_Liq = ('Fo_Liq','Liquid','Forsterite_xmelts')

Now we write down the SymPy expression for the reaction. This, along with the list of reactants, products, parameters, units and variable symbols are passed into the 'add_reaction_to_model' function. This function also requires a 'name' field, which should be consistent with the variable assigned to the SymPy expression; for example, here our expression is Fo_melting = ..., so our name is 'Fo_melting'.

In [ ]:

# SymPy expression for reaction rate
r = sym.exp(-T0/T)
rp = r
rm = r

S0p = sym.sqrt(Phi[iOl])
S0m = sym.sqrt(Phi[iLiq])
Fo_melting = sym.Piecewise((rp*S0p*A[j]/R/T,A[j]>=0),(rm*S0m*A[j]/R/T,A[j]<0),(0,True))
Fo_melting

In [ ]:
reactants = [ Fo_Ol]
products = [ Fo_Liq]
rxn.add_reaction_to_model('Fo_melting', reactants, products, Fo_melting, params)

### Do the same for the Fayalite melting reaction

In [ ]:
# Reaction 1
j = 1

# Reactants
Fa_Ol = ('Fa_Ol','Olivine','Fayalite_berman')

# Products
Fa_Liq = ('Fa_Liq','Liquid','Fayalite_xmelts')

# SymPy expression for reaction rate
r = sym.exp(-T0/T)
rp = r
rm = r

S1p = sym.sqrt(Phi[iOl])
S1m = sym.sqrt(Phi[iLiq])
Fa_melting = sym.Piecewise((rp*S1p*A[j]/R/T,A[1]>=0),(rm*S1m*A[j]/R/T,A[j]<0),(0,True))
Fa_melting

In [ ]:
reactants = [ Fa_Ol ] 
products = [ Fa_Liq ]
rxn.add_reaction_to_model('Fa_melting', reactants, products, Fa_melting, params)

### The model_dict has now been updated to contain all of the information for these reactions

In [ ]:
rxn.model_dict

Return a dictionary of settable name value pairs

In [ ]:
values_dict = rxn.get_values()
values_dict

Update some of these values...


In [ ]:
values_dict.update(dict(name='fo_fa_binary',
                        reference=reference,
                        T0=1000.,
                        R=8.314))
values_dict

... and update them in the model_dict using the 'set_values' function

In [ ]:
rxn.set_values(values_dict)
rxn.model_dict

### Generate Spud XML files

Set some directories

In [ ]:
HOME_DIR = os.path.abspath(os.curdir)
SPUD_DIR = HOME_DIR+'/../reactions'

try:
    os.mkdir(SPUD_DIR)
except:
    pass

### Dump the spud file

In [ ]:
rxn.to_xml(path=SPUD_DIR)